#  Week 26 Hacks
> hacks for unit 2.4a
- toc: true
- type: ap
- week: 26

# week 26 hacks

## In the Blog add notes and observations on each code cell.

### cell block number 1

In [1]:
# Import necessary modules
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

# Create a Flask application object
app = Flask(__name__)

# Specify the location of the database and configure SQLAlchemy
database = 'sqlite:///files/sqlite.db'  # path and filename of database
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy()

# Initialize the SQLAlchemy app with the Flask app
db.init_app(app)


### cell block number 2

In [ ]:
# Import necessary modules for the User class
import datetime
from datetime import datetime
import json

from sqlalchemy.exc import IntegrityError
from werkzeug.security import generate_password_hash, check_password_hash


# Define the User class to manage actions in the 'users' table
class User(db.Model):
    __tablename__ = 'users'  # Set the name of the table to 'users'

    # Define the User schema with columns for the table
    id = db.Column(db.Integer, primary_key=True)  # Column for the primary key (integer)
    _name = db.Column(db.String(255), unique=False, nullable=False)  # Column for name (string)
    _uid = db.Column(db.String(255), unique=True, nullable=False)  # Column for user id (string)
    _password = db.Column(db.String(255), unique=False, nullable=False)  # Column for password (string)
    _dob = db.Column(db.Date)  # Column for date of birth (date)

    # Constructor for the User object, initializes instance variables
    def __init__(self, name, uid, password="123qwerty", dob=datetime.today()):
        self._name = name
        self._uid = uid
        self.set_password(password)
        if isinstance(dob, str):  # Check if dob is a string, set to today's date if true
            dob = date=datetime.today()
        self._dob = dob

    # Getter method for the name variable
    @property
    def name(self):
        return self._name
    
    # Setter method for the name variable
    @name.setter
    def name(self, name):
        self._name = name
    
    # Getter method for the uid variable
    @property
    def uid(self):
        return self._uid
    
    # Setter method for the uid variable
    @uid.setter
    def uid(self, uid):
        self._uid = uid
        
    # Check if uid parameter matches user id in object, return boolean
    def is_uid(self, uid):
        return self._uid == uid
    
    # Getter method for the password variable, only shows the first 10 characters
    @property
    def password(self):
        return self._password[0:10] + "..."

    # Setter method for the password variable, encrypts the password using SHA256 hashing algorithm
    def set_password(self, password):
        """Create a hashed password."""
        self._password = generate_password_hash(password, method='sha256')

    # Check if password parameter matches stored/encrypted password, return boolean
    def is_password(self, password):
        """Check against hashed password."""
        result = check_password_hash(self._password, password)
        return result
    
    # Getter method for the dob variable, returns the dob as a formatted string
    @property
    def dob(self):
        dob_string = self._dob.strftime('%m-%d-%Y')
        return dob_string
    
    # Setter method for the dob variable, sets the dob to today's date if it is a string
    @dob.setter
    def dob(self, dob):
        if isinstance(dob, str):     
            dob = date=datetime.today()
        self._dob = dob
    
    # Getter method for the age variable, calculates the age based on dob
    @property
    def age(self):
        today = datetime.today()
        return today.year - self._dob.year - ((today.month, today.day) < (self._dob.month, self._dob.day))
    
    # Convert the User object to a dictionary
    def read(self):
        return {
            "id": self.id,
            "name": self.name

### code block number 3

In [ ]:
# Database Creation and Testing

# Builds working data for testing
def initUsers():
    with app.app_context():
        # Create database and tables
        db.create_all()

        # Add tester data to table
        u1 = User(name='Thomas Edison', uid='toby', password='123toby', dob=datetime(1847, 2, 11))
        u2 = User(name='Nikola Tesla', uid='niko', password='123niko')
        u3 = User(name='Alexander Graham Bell', uid='lex', password='123lex')
        u4 = User(name='Eli Whitney', uid='whit', password='123whit')
        u5 = User(name='Indiana Jones', uid='indi', dob=datetime(1920, 10, 21))
        u6 = User(name='Marion Ravenwood', uid='raven', dob=datetime(1921, 10, 21))

        # Add users to a list
        users = [u1, u2, u3, u4, u5, u6]

        # Build sample user/note(s) data
        for user in users:
            try:
                # Add user to table
                object = user.create()
                print(f"Created new uid {object.uid}")
            except:
                # Handle error if object is not created
                print(f"Records exist uid {user.uid}, or error.")
                
# Call initUsers function to create and populate the database
initUsers()


### code block number 4

In [ ]:

# Builds working data for testing
def initUsers():
    with app.app_context():
        """Create database and tables"""
        db.create_all()
        """Tester data for table"""
        u1 = User(name='Thomas Edison', uid='toby', password='123toby', dob=datetime(1847, 2, 11))
        u2 = User(name='Nikola Tesla', uid='niko', password='123niko')
        u3 = User(name='Alexander Graham Bell', uid='lex', password='123lex')
        u4 = User(name='Eli Whitney', uid='whit', password='123whit')
        u5 = User(name='Indiana Jones', uid='indi', dob=datetime(1920, 10, 21))
        u6 = User(name='Marion Ravenwood', uid='raven', dob=datetime(1921, 10, 21))


        users = [u1, u2, u3, u4, u5, u6]

        """Builds sample user/note(s) data"""
        for user in users:
            try:
                '''add user to table'''
                object = user.create()
                print(f"Created new uid {object.uid}")
            except:  # error raised if object nit created
                '''fails with bad or duplicate data'''
                print(f"Records exist uid {user.uid}, or error.")
                
initUsers() 

### code block number 5


In [ ]:
# SQLAlchemy extracts single user from database matching User ID
def find_by_uid(uid):
    with app.app_context():
        user = User.query.filter_by(_uid=uid).first()
    return user # returns user object

# Check credentials by finding user and verify password
def check_credentials(uid, password):
    # query email and return user record
    user = find_by_uid(uid)
    if user == None:
        return False
    if (user.is_password(password)):
        return True
    return False
        
#check_credentials("indi", "123qwerty")

### code block number 6


In [ ]:
# Inputs, Try/Except, and SQLAlchemy work together to build a valid database object
def create():
    # optimize user time to see if uid exists
    uid = input("Enter your user id:")
    user = find_by_uid(uid)
    try:
        print("Found\n", user.read())
        return
    except:
        pass # keep going
    
    # request value that ensure creating valid object
    name = input("Enter your name:")
    password = input("Enter your password")
    
    # Initialize User object before date
    user = User(name=name, 
                uid=uid, 
                password=password
                )
    
    # create user.dob, fail with today as dob
    dob = input("Enter your date of birth 'YYYY-MM-DD'")
    try:
        user.dob = datetime.strptime(dob, '%Y-%m-%d').date()
    except ValueError:
        user.dob = datetime.today()
        print(f"Invalid date {dob} require YYYY-mm-dd, date defaulted to {user.dob}")
           
    # write object to database
    with app.app_context():
        try:
            object = user.create()
            print("Created\n", object.read())
        except:  # error raised if object not created
            print("Unknown error uid {uid}")
        
create()

----

## Change blog to your own database.

In [2]:
"""
These imports define the key objects
"""

from flask import Flask
from flask_sqlalchemy import SQLAlchemy

"""
These object and definitions are used throughout the Jupyter Notebook.
"""

# Setup of key Flask object (app)
app = Flask(__name__)
# Setup SQLAlchemy object and properties for the database (db)
database = 'sqlite:///sqlite.db'  # path and filename of database
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy()


# This belongs in place where it runs once per project
db.init_app(app)

""" database dependencies to support sqlite examples """
import datetime
from datetime import datetime
import json

from sqlalchemy.exc import IntegrityError
from werkzeug.security import generate_password_hash, check_password_hash


''' Tutorial: https://www.sqlalchemy.org/library.html#tutorials, try to get into a Python shell and follow along '''

# Define the User class to manage actions in the 'users' table
# -- Object Relational Mapping (ORM) is the key concept of SQLAlchemy
# -- a.) db.Model is like an inner layer of the onion in ORM
# -- b.) User represents data we want to store, something that is built on db.Model
# -- c.) SQLAlchemy ORM is layer on top of SQLAlchemy Core, then SQLAlchemy engine, SQL
class User(db.Model):
    __tablename__ = 'users'  # table name is plural, class name is singular

    # Define the User schema with "vars" from object
    id = db.Column(db.Integer, primary_key=True)
    _dsname = db.Column(db.String(255), unique=False, nullable=False)
    _dsad = db.Column(db.String(255), unique=True, nullable=False)


    # constructor of a User object, initializes the instance variables within object (self)
    def __init__(self, dsname, dsad):
        self._dsname = dsname    # variables with self prefix become part of the object, 
        self._dsad = dsad
 

    # a name getter method, extracts name from object
    @property
    def dsname(self):
        return self._dsname
    
    # a setter function, allows name to be updated after initial object creation
    @dsname.setter
    def dsname(self, dsname):
        self._dsname = dsname
    
    # a getter method, extracts uid from object
    @property
    def dsad(self):
        return self._dsad
    
    # a setter function, allows uid to be updated after initial object creation
    @dsad.setter
    def dsad(self, dsad):
        self._dsad = dsad
        
    # check if uid parameter matches user id in object, return boolean
    def is_dsad(self, dsad):
        return self._dsad == dsad
    

    
    # output content using str(object) is in human readable form
    # output content using json dumps, this is ready for API response
    def __str__(self):
        return json.dumps(self.read())

    # CRUD create/add a new record to the table
    # returns self or None on error
    def create(self):
        try:
            # creates a person object from User(db.Model) class, passes initializers
            db.session.add(self)  # add prepares to persist person object to Users table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            db.session.remove()
            return None

    # CRUD read converts self to dictionary
    # returns dictionary
    def read(self):
        return {
            "id": self.id,
            "dsname": self.dsname,
            "dsad": self.dsad,
        }

    # CRUD update: updates user name, password, phone
    # returns self
    def update(self, name="", dsad=""):
        """only updates values with length"""
        if len(dsname) > 0:
            self.dsname = dsname
        if len(dsad) > 0:
            self.dsad = dsad

    # delete crud: deletes user
    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None
    
"""Database Creation and Testing """


# Builds working data for testing
def initUsers():
    with app.app_context():
        """Create database and tables"""
        db.create_all()
        """Tester data for table"""
        u1 = User(dsname='Thomas Edison', dsad='toby')
        u2 = User(dsname='Nikola Tesla', dsad='niko')
        u3 = User(dsname='Alexander Graham Bell', dsad='lex')
        u4 = User(dsname='Eli Whitney', dsad='whit')
        u5 = User(dsname='Indiana Jones', dsad='indi')
        u6 = User(dsname='Marion Ravenwood', dsad='raven')


        users = [u1, u2, u3, u4, u5, u6]

        """Builds sample user/note(s) data"""
        for user in users:
            try:
                '''add user to table'''
                object = user.create()
                print(f"Created new dsad {object.dsad}")
            except:  # error raised if object nit created
                '''fails with bad or duplicate data'''
                print(f"Records exist dsad {user.dsad}, or error.")
                
initUsers() 

# SQLAlchemy extracts single user from database matching User ID
def find_by_dsad(dsad):
    with app.app_context():
        user = User.query.filter_by(_dsad=dsad).first()
    return user # returns user object

        
#check_credentials("indi", "123qwerty")

# Inputs, Try/Except, and SQLAlchemy work together to build a valid database object
def create():
    # optimize user time to see if uid exists
    dsad = input("Enter your user id:")
    user = find_by_dsad(dsad)
    try:
        print("Found\n", user.read())
        return
    except:
        pass # keep going
    
    # request value that ensure creating valid object
    dsname = input("Enter your dealership name:")
    password = input("Enter your password")
    
    # Initialize User object before date
    user = User(dsname=dsname, 
                dsad=dsad, 
                )
    
           
    # write object to database
    with app.app_context():
        try:
            object = user.create()
            print("Created\n", object.read())
        except:  # error raised if object not created
            print("Unknown error dsad {dsad}")
        
create()


# SQLAlchemy extracts all users from database, turns each user into JSON
def read():
    with app.app_context():
        table = User.query.all()
    json_ready = [user.read() for user in table] # "List Comprehensions", for each user add user.read() to list
    return json_ready

read()

Records exist dsad toby, or error.
Records exist dsad niko, or error.
Records exist dsad lex, or error.
Records exist dsad whit, or error.
Records exist dsad indi, or error.
Records exist dsad raven, or error.


OperationalError: (sqlite3.OperationalError) no such column: users._dsname
[SQL: SELECT users.id AS users_id, users._dsname AS users__dsname, users._dsad AS users__dsad 
FROM users 
WHERE users._dsad = ?
 LIMIT ? OFFSET ?]
[parameters: ('19', 1, 0)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

----

## understand debugging 

![image](_notebooks/images/debugging.png)

# unit 2.4b hacks

## observations

### code block 1

In [ ]:
# this code block connect to make the databse and connect to the databse. 

import sqlite3

database = 'instance/sqlite.db' # this is location of database

def schema():
    
    # Connect to the database file
    conn = sqlite3.connect(database)

    # Create a cursor object to execute SQL queries
    cursor = conn.cursor()
    
    # Fetch results of Schema
    results = cursor.execute("PRAGMA table_info('users')").fetchall()

    # Print the results
    for row in results:
        print(row)

    # Close the database connection
    conn.close()
    
schema()


### code block 2

In [ ]:
# reads the databse

import sqlite3

def read():
    # Connect to the database file
    conn = sqlite3.connect(database)

    # Create a cursor object to execute SQL queries
    cursor = conn.cursor()
    
    # Execute a SELECT statement to retrieve data from a table
    results = cursor.execute('SELECT * FROM users').fetchall()

    # Print the results
    if len(results) == 0:
        print("Table is empty")
    else:
        for row in results:
            print(row)

    # Close the cursor and connection objects
    cursor.close()
    conn.close()
    
read()


### code block 3

In [ ]:
import sqlite3

def create():
    name = input("Enter your name:")
    uid = input("Enter your user id:")
    password = input("Enter your password")
    dob = input("Enter your date of birth 'YYYY-MM-DD'")
    
    # Connect to the database file
    conn = sqlite3.connect(database)

    # Create a cursor object to execute SQL commands
    cursor = conn.cursor()

    try:
        # Execute an SQL command to insert data into a table
        cursor.execute("INSERT INTO users (_name, _uid, _password, _dob) VALUES (?, ?, ?, ?)", (name, uid, password, dob))
        
        # Commit the changes to the database
        conn.commit()
        print(f"A new user record {uid} has been created")
                
    except sqlite3.Error as error:
        print("Error while executing the INSERT:", error)


    # Close the cursor and connection objects
    cursor.close()
    conn.close()
    
#create()

### code block 4

In [ ]:
import sqlite3

def update():
    uid = input("Enter user id to update")
    password = input("Enter updated password")
    if len(password) < 2:
        message = "hacked"
        password = 'gothackednewpassword123'
    else:
        message = "successfully updated"

    # Connect to the database file
    conn = sqlite3.connect(database)

    # Create a cursor object to execute SQL commands
    cursor = conn.cursor()

    try:
        # Execute an SQL command to update data in a table
        cursor.execute("UPDATE users SET _password = ? WHERE _uid = ?", (password, uid))
        if cursor.rowcount == 0:
            # The uid was not found in the table
            print(f"No uid {uid} was not found in the table")
        else:
            print(f"The row with user id {uid} the password has been {message}")
            conn.commit()
    except sqlite3.Error as error:
        print("Error while executing the UPDATE:", error)
        
    
    # Close the cursor and connection objects
    cursor.close()
    conn.close()
    
#update()


### code block 5

In [ ]:
import sqlite3

def delete():
    uid = input("Enter user id to delete")

    # Connect to the database file
    conn = sqlite3.connect(database)

    # Create a cursor object to execute SQL commands
    cursor = conn.cursor()
    
    try:
        cursor.execute("DELETE FROM users WHERE _uid = ?", (uid,))
        if cursor.rowcount == 0:
            # The uid was not found in the table
            print(f"No uid {uid} was not found in the table")
        else:
            # The uid was found in the table and the row was deleted
            print(f"The row with uid {uid} was successfully deleted")
        conn.commit()
    except sqlite3.Error as error:
        print("Error while executing the DELETE:", error)
        
    # Close the cursor and connection objects
    cursor.close()
    conn.close()
    
#delete()

### code block 6


In [ ]:
# Menu, to run other cells from one control point
def menu():
    operation = input("Enter: (C)reate (R)ead (U)pdate or (D)elete or (S)chema")
    if operation.lower() == 'c':
        create()
    elif operation.lower() == 'r':
        read()
    elif operation.lower() == 'u':
        update()
    elif operation.lower() == 'd':
        delete()
    elif operation.lower() == 's':
        schema()
    elif len(operation)==0: # Escape Key
        return
    else:
        print("Please enter c, r, u, or d") 
    menu() # recursion, repeat menu
        
try:
    menu() # start menu
except:
    print("Perform Jupyter 'Run All' prior to starting menu")


## in this implementation, do you see procedural abstraction?

yes, in this implementation, i do see a procedural abstraction. using procedural abstraction, the code can write code that is easier to read and maintain. unlike the previous code, this did not give me a headache. 